In [ ]:
### import pickle
import tensorflow as tf
import numpy as np

from keras.layers import Lambda,Reshape,concatenate,Input, Embedding, LSTM
from keras.layers import Dense,Dropout, Activation ,Flatten ,RepeatVector, Bidirectional,GlobalAveragePooling1D
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras.activations import softmax
from keras import regularizers

from keras import backend as K, regularizers, constraints, initializers
from keras.engine.topology import Layer

from keras.layers import merge
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from keras.models import Sequential

from Attention import Attention

from keras.layers import Concatenate,Dot
from keras.layers import Permute, merge

# FOR ATAE
from AttentionwithContext import AttentionWithContext
from Final import FinalSentenceRepresentation

from final2 import Final2

from keras.callbacks import ModelCheckpoint


In [2]:
a=pickle.load(open("all_data.dat","rb"))

input_i=a['sentence']
input_t=a['target']

input_t_IAN=a['target_for_IAN']

embedding_matrix=a['embedding_matrix']

sentiment= a['train_sentiment']

v_sentence=a['v_sentence']
v_target= a['v_target']
v_sentiment= a['v_sentiment']
v_target_IAN =a['v_target_IAN']

vocab_size= a['vocab_size']
aspect=a['aspect']
aspect_y=a['v_aspect']

In [3]:
len(aspect_y)

192

In [4]:
a.keys()

dict_keys(['embedding_matrix', 'v_sentence', 'vocab_size', 'v_target_IAN', 'v_aspect', 'v_target', 'aspect', 'target_for_IAN', 'train_sentiment', 'target', 'sentence', 'v_sentiment'])

In [5]:
input_t_IAN[10:20]

array([[411, 292,   0,   0,   0],
       [  2,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0],
       [853,   0,   0,   0,   0],
       [ 77,   0,   0,   0,   0],
       [ 33,   0,   0,   0,   0],
       [ 86,   0,   0,   0,   0],
       [857,   0,   0,   0,   0],
       [859,   0,   0,   0,   0],
       [357,   0,   0,   0,   0]])

In [6]:
def sk_mse(y_true,y_pred):
     return K.mean(K.square(y_pred - y_true), axis=-1)

In [7]:
max_length=11

### Inputs for  models

In [8]:
input_i

array([[ 442,   14, 1457, ...,  840,   31, 1460],
       [ 288,   28,  289, ...,    0,    0,    0],
       [1461,  219,   37, ...,    0,    0,    0],
       ...,
       [1426, 1265, 1213, ...,    0,    0,    0],
       [3320, 1163, 3321, ...,    0,    0,    0],
       [1240, 1337, 3323, ...,    0,    0,    0]])

In [9]:
input_context=input_i
input_target=input_t

In [10]:
#len(input_context[0])

In [11]:
input_target[0]

array([14])

In [12]:
v_target[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
#len(input_target[0])

In [14]:
target = np.tile(input_t,11)
input_target= target


In [15]:
input_target[0]


array([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14])

In [16]:
trainX= [input_context, input_target]
testX= [v_sentence,v_target]
testY=v_sentiment
trainY= sentiment

In [17]:
max_length=11
#max_target_len=11

In [18]:
#i_less= input_context[:100]
#t_less = input_target[:100]
#trainY_less= trainY[:100]

#t_less[0]

In [19]:
#test_X=[v_sentence[:10],v_target[:10]]
#test_Y=v_sentiment[:10]

# Sentiment Models

# AT MODEL

In [20]:
def Model_AT(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim):
    
    
    input_context= Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(max_length,),name='Target')
  
   
    #print(input_target.shape)
    #print(input_context.shape)
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    v_target = embedding(input_target)
    
    
    #print(context.shape)
    print("Target" ,v_target.shape)

    
    #concat = concatenate(inputs =[target,context])
    #print(concat.shape)
    #inputs = Dropout(dropout)(concat)
    #print(inputs.shape)   
    
    
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(context)
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(H)
    H_all, H_last , _ = LSTM (lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="AT")(H)

    #print(H_all.shape , "H_all before reduction")
    H_all_1 =GlobalAveragePooling1D()(H_all)
    #print(H_all.shape , "H_all after reduction")

    #Target = GlobalAveragePooling1D()(v_target)
    #print(Target.shape , "Target after reduction")
    
    concat = concatenate(inputs =[v_target,H_all])
    print("Concat",concat.shape)
    
    r=AttentionWithContext(name='Attention')([H_all,concat])
    
    #print("h_ n ", H_last.shape)
    

    out=FinalSentenceRepresentation(name='Final')([r , H_all_1])
    
    #out=Dense(int((2*lstm_out+1)/2),activation='relu')(out)
            
    out= Dense(1, activation='sigmoid',  kernel_regularizer=regularizers.l2(0.01))(out)

    #print(out.shape)
    
    AT_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    AT_model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = [sk_mse])
    
    
    print(AT_model.summary())
    
    return AT_model

In [21]:
AT_model = Model_AT(learning_rate=0.00069,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

Target (?, 11, 300)
Concat (?, 11, 600)
M1 (?, 11, 600)
M (?, 11, 1)
alpha (?, 11, 1)
r (?, 300)
H_last (?, 300)
m1  (?, 300)
m2.shape (?, 300)
(?, 300) h_final
OUT (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 11, 300)      997200      Context[0][0]                    
                                                                 Target[0][0]                     
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 11, 300)      721200      embedding_1[0][0]                
____________________________________

In [22]:
AT_model = AT_model.fit(x=trainX,y=trainY, epochs=50,batch_size=64,validation_data=(testX,testY))



Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 8s 7ms/step - loss: 0.6996 - sk_mse: 0.0409 - val_loss: 0.7112 - val_sk_mse: 0.0483
Epoch 2/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.6912 - sk_mse: 0.0369 - val_loss: 0.7094 - val_sk_mse: 0.0474
Epoch 3/50
1173/1173 [==============================] - 5s 5ms/step - loss: 0.6824 - sk_mse: 0.0326 - val_loss: 0.7064 - val_sk_mse: 0.0461
Epoch 4/50
1173/1173 [==============================] - 5s 5ms/step - loss: 0.6774 - sk_mse: 0.0303 - val_loss: 0.7036 - val_sk_mse: 0.0447
Epoch 5/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.6728 - sk_mse: 0.0282 - val_loss: 0.7023 - val_sk_mse: 0.0443
Epoch 6/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.6683 - sk_mse: 0.0262 - val_loss: 0.7059 - val_sk_mse: 0.0460
Epoch 7/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.6678 - sk_mse: 0.0260 - val_loss: 0

# Adding Attention to our model 

In [24]:
def define_model_1(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,em_dim):
     
    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
        
    a = Bidirectional(LSTM(300, return_sequences=True,recurrent_dropout=dropout))(context)
    
    print(a.shape)

    alpha = Attention()(a)
    
    x=Dense(300,activation='relu')(alpha)
        
    out=Dense(1,activation='sigmoid')(x)
    
    model= Model(inputs=input_context ,outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss = 'mse', optimizer=optimizer,metrics = ['mae'])
    
    
    print(model.summary())
    
    return model

In [25]:
Our_model = define_model_1(learning_rate=0.001,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

(?, ?, 600)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Context (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 11, 300)           997200    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 11, 600)           1442400   
_________________________________________________________________
attention_1 (Attention)      (None, 600)               611       
_________________________________________________________________
dense_2 (Dense)              (None, 300)               180300    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 2,620,812
Trainable params: 1,623,612
Non-trainable params: 997,200
________________________________________________

In [26]:
Our_model =Our_model.fit(x=input_context,y=trainY, epochs=50,batch_size=64)


Epoch 1/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.0400 - mean_absolute_error: 0.1737
Epoch 2/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0346 - mean_absolute_error: 0.1584
Epoch 3/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0286 - mean_absolute_error: 0.1389
Epoch 4/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0285 - mean_absolute_error: 0.1391
Epoch 5/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0239 - mean_absolute_error: 0.1239
Epoch 6/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0219 - mean_absolute_error: 0.1186
Epoch 7/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0182 - mean_absolute_error: 0.1066
Epoch 8/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0160 - mean_absolute_error: 0.0992
Epoch 9/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0155 - mean_absolu

# AT-AE MODEL


In [29]:
def Model_ATAE(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim):
    
    
    input_context= Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(max_length,),name='Target')
  
   
    #print(input_target.shape)
    #print(input_context.shape)
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    v_target = embedding(input_target)
    
    #print(context.shape)
    print("Target" ,v_target.shape)

    
    concat = concatenate(inputs =[v_target,context])
    print(concat.shape)
    inputs = Dropout(dropout)(concat)
    print(inputs.shape)   
    
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(inputs)
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(H)
    H_all, H_last , _ = LSTM (lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="ATAE")(H)


    H_all_1 = GlobalAveragePooling1D()(H_all)
    print(H_all.shape , "H_all after reduction")
    
    concat = concatenate(inputs =[v_target,H_all])
    print("Concat",concat.shape)
    
    r=AttentionWithContext(name='Attention')([H_all,concat])
    
    #print("h_ n ", H_last.shape)
    
    
    out=FinalSentenceRepresentation(name='Final')([r , H_all_1])

    out= Dense(1, activation='sigmoid',  kernel_regularizer=regularizers.l2(0.01))(out)

    #print(out.shape)
    
    ATAE_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    ATAE_model.compile(optimizer = optimizer, loss = 'mse', metrics = ['mae'])
    
    
    print(ATAE_model.summary())
    
    return ATAE_model

In [30]:
ATAE_model = Model_ATAE(learning_rate=0.001,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

Target (?, 11, 300)
(?, 11, 600)
(?, 11, 600)
(?, ?, 300) H_all after reduction
Concat (?, 11, 600)
M1 (?, 11, 600)
M (?, 11, 1)
alpha (?, 11, 1)
r (?, 300)
H_last (?, 300)
m1  (?, 300)
m2.shape (?, 300)
(?, 300) h_final
OUT (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Target (InputLayer)             (None, 11)           0                                            
__________________________________________________________________________________________________
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 11, 300)      997200      Context[0][0]                    
                                                                 Target[0][

In [31]:
ATAE_model = ATAE_model.fit(x=trainX,y=trainY, epochs=50,batch_size=64,validation_data=(testX,testY))


Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 10s 8ms/step - loss: 0.0628 - mean_absolute_error: 0.1714 - val_loss: 0.0688 - val_mean_absolute_error: 0.1748
Epoch 2/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.0585 - mean_absolute_error: 0.1605 - val_loss: 0.0702 - val_mean_absolute_error: 0.1753
Epoch 3/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.0553 - mean_absolute_error: 0.1500 - val_loss: 0.0670 - val_mean_absolute_error: 0.1739
Epoch 4/50
1173/1173 [==============================] - 7s 6ms/step - loss: 0.0535 - mean_absolute_error: 0.1465 - val_loss: 0.0655 - val_mean_absolute_error: 0.1740
Epoch 5/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.0510 - mean_absolute_error: 0.1422 - val_loss: 0.0656 - val_mean_absolute_error: 0.1717
Epoch 6/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.0500 - mean_absolute_error: 0.1389 - val_los

Epoch 50/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.0192 - mean_absolute_error: 0.0865 - val_loss: 0.0578 - val_mean_absolute_error: 0.1840


 <font color='red'>


loss: 0.0192 - mean_absolute_error: 0.0865
</font> 

In [117]:
ATAE_model.save('ATAE_model.h5')


# AE

In [35]:

def Model_AE(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim):
     
    input_context= Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(max_length,),name='Target')
  
   
    #print(input_target.shape)
    #print(input_context.shape)
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    v_target = embedding(input_target)
    
    #print(context.shape)
    print("Target" ,v_target.shape)

    
    concat = concatenate(inputs =[v_target,context])
    print(concat.shape)
    inputs = Dropout(dropout)(concat)
    print(inputs.shape)   
    
    H_last = Bidirectional(LSTM (lstm_out, recurrent_dropout=dropout,name="ATAE"))(inputs)
    
    x=Dense(300,activation='relu')(H_last)
        
    out=Dense(1,activation='sigmoid')(x)
    
    #print(out.shape)
    
    AE_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    AE_model.compile(optimizer = optimizer, loss = 'mse', metrics = ['mae'])
    
    
    print(AE_model.summary())
    
    return AE_model

In [36]:

AE_model = Model_AE(learning_rate=0.001,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)


Target (?, 11, 300)
(?, 11, 600)
(?, 11, 600)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Target (InputLayer)             (None, 11)           0                                            
__________________________________________________________________________________________________
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 11, 300)      997200      Context[0][0]                    
                                                                 Target[0][0]                     
__________________________________________________________________________________________________
concatenate_6 (Concatenate)     (None, 11, 600)      0         

In [37]:
AE_model = AE_model.fit(x=trainX,y=trainY, epochs=50,batch_size=64,validation_data=(testX,testY))



Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.0418 - mean_absolute_error: 0.1735 - val_loss: 0.0438 - val_mean_absolute_error: 0.1739
Epoch 2/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.0346 - mean_absolute_error: 0.1582 - val_loss: 0.0459 - val_mean_absolute_error: 0.1762
Epoch 3/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.0302 - mean_absolute_error: 0.1425 - val_loss: 0.0450 - val_mean_absolute_error: 0.1754
Epoch 4/50
1173/1173 [==============================] - 4s 3ms/step - loss: 0.0276 - mean_absolute_error: 0.1354 - val_loss: 0.0459 - val_mean_absolute_error: 0.1768
Epoch 5/50
1173/1173 [==============================] - 4s 3ms/step - loss: 0.0260 - mean_absolute_error: 0.1301 - val_loss: 0.0465 - val_mean_absolute_error: 0.1752
Epoch 6/50
1173/1173 [==============================] - 4s 3ms/step - loss: 0.0240 - mean_absolute_error: 0.1233 - val_loss

Epoch 50/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0042 - mean_absolute_error: 0.0473 - val_loss: 0.0564 - val_mean_absolute_error: 0.1911


In [ ]:
AE_model.save('AE_model.h5')



# IAN

In [108]:

concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "tanh")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [109]:
def one_step_attention(h, avg,shape):
    
    avg = RepeatVector(shape)(avg)
    #print("avg",avg.shape)
    
    concat = concatenator([h, avg])
    #print("concat",concat.shape)
    
    e = densor(concat)
    #print("e",e.shape)

    alphas = activator(e)
    #print("alphas",alphas.shape)

    context = dotor([alphas, h])
    #print("context",context.shape)
    
    return K.sum(context, axis=1)

In [111]:

def Model_IAN(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim):
    
    input_context = Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(5,),name='Target')
    
    print("Target ",input_target.shape)
    print("Context" ,input_context.shape)
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    embedding_T=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=5,trainable = em_trainable_flag)

    context=embedding(input_context)
    target= embedding_T(input_target)
    print("Target_embedding",target.shape)
    print("Context_embedding", context.shape)
 
    H_c , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_C")(context)
    H_t , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_T")(target)

    print("hc", H_c.shape)
    print("ht",H_t.shape)
    
    c_avg = GlobalAveragePooling1D(name='POOL_C')(H_c)  
    t_avg = GlobalAveragePooling1D(name='POOL_T')(H_t)
    
    print("C_AVG", c_avg.shape)
    print("t_avg",t_avg.shape)
    

    c_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_c,t_avg])
    t_r = Lambda(lambda x: one_step_attention(x[0],x[1],5))([H_t,c_avg])
    
    print("c_r",c_r.shape)
    print("t_r",t_r.shape)
   
    d = concatenate(inputs=[c_r , t_r])
    
    print("d",d.shape)
    
    out= Dense(300, activation='relu')(d)

    
    out= Dense(1, activation='sigmoid',W_regularizer=regularizers.l2(0.001))(out)


    print("out", out.shape)
    
    IAN_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    
    IAN_model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    print( IAN_model.summary())
    
    return  IAN_model
   


In [108]:
IAN_model=Model_IAN(learning_rate=0.01,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

Target  (?, 5)
Context (?, 11)
Target_embedding (?, 5, 300)
Context_embedding (?, 11, 300)
hc (?, ?, 300)
ht (?, ?, 300)
C_AVG (?, 300)
t_avg (?, 300)
c_r (?, 300)
t_r (?, 300)
d (?, 600)
out (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
Target (InputLayer)             (None, 5)            0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 11, 300)      997200      Context[0][0]                    
__________________________________________________________________________________________________
embedding

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_regularizer=<keras.reg...)`


In [109]:
trainX_IAN= [input_context,input_t_IAN]
testX_IAN=[v_sentence,v_target_IAN]

In [110]:
IAN_model = IAN_model.fit(x=trainX_IAN,y=trainY, epochs=50,batch_size=64,validation_data=(testX_IAN,testY))



Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 10s 8ms/step - loss: 0.2205 - mean_absolute_error: 0.4191 - val_loss: 0.2868 - val_mean_absolute_error: 0.4898
Epoch 2/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.2341 - mean_absolute_error: 0.4391 - val_loss: 0.2861 - val_mean_absolute_error: 0.4898
Epoch 3/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.2335 - mean_absolute_error: 0.4391 - val_loss: 0.2857 - val_mean_absolute_error: 0.4898
Epoch 4/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.2332 - mean_absolute_error: 0.4391 - val_loss: 0.2854 - val_mean_absolute_error: 0.4898
Epoch 5/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.2330 - mean_absolute_error: 0.4391 - val_loss: 0.2853 - val_mean_absolute_error: 0.4898
Epoch 6/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.2328 - mean_absolute_error: 0.4391 - val_los

Epoch 50/50
1173/1173 [==============================] - 4s 4ms/step - loss: 0.2421 - mean_absolute_error: 0.4391 - val_loss: 0.2945 - val_mean_absolute_error: 0.4898


In [ ]:
IAN_model.save('IAN_model.h5')

 <font color='red'>
After 50 epochs :
loss: 0.2421 - mean_absolute_error: 0.4391 - val_loss: 0.2945 - val_mean_absolute_error: 0.4898
    </font>

# IAN with One word target

In [104]:

def Model_IAN_1(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim):
    
    input_context = Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(11,),name='Target')
    
    print("Target ",input_target.shape)
    print("Context" ,input_context.shape)
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)

    context=embedding(input_context)
    target= embedding(input_target)
    print("Target_embedding",target.shape)
    print("Context_embedding", context.shape)
 
    H_c , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_C")(context)
    H_t , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_T")(target)

    print("hc", H_c.shape)
    print("ht",H_t.shape)
    
    c_avg = GlobalAveragePooling1D(name='POOL_C')(H_c)  
    t_avg = GlobalAveragePooling1D(name='POOL_T')(H_t)
    
    print("C_AVG", c_avg.shape)
    print("t_avg",t_avg.shape)
    

    c_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_c,t_avg])
    t_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_t,c_avg])
    
    print("c_r",c_r.shape)
    print("t_r",t_r.shape)
   
    d = concatenate(inputs=[c_r , t_r])
    
    print("d",d.shape)
    
    out= Dense(300, activation='relu')(d)

    
    out= Dense(1, activation='sigmoid',W_regularizer=regularizers.l2(0.001))(out)


    print("out", out.shape)
    
    IAN_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    
    IAN_model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    print( IAN_model.summary())
    
    return  IAN_model
   


In [105]:
IAN_model_1=Model_IAN_1(learning_rate=0.01,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

Target  (?, 11)
Context (?, 11)
Target_embedding (?, 11, 300)
Context_embedding (?, 11, 300)
hc (?, ?, 300)
ht (?, ?, 300)
C_AVG (?, 300)
t_avg (?, 300)
c_r (?, 300)
t_r (?, 300)
d (?, 600)
out (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
Target (InputLayer)             (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_32 (Embedding)        (None, 11, 300)      997200      Context[0][0]                    
                                                                 Target[0][0]                     
_______

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_regularizer=<keras.reg...)`


In [106]:
IAN_model_1 = IAN_model_1.fit(x=trainX,y=trainY, epochs=50,batch_size=64,validation_data=(testX,testY))


Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 10s 8ms/step - loss: 0.0492 - mean_absolute_error: 0.1824 - val_loss: 0.0499 - val_mean_absolute_error: 0.1818
Epoch 2/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0378 - mean_absolute_error: 0.1585 - val_loss: 0.0502 - val_mean_absolute_error: 0.1811
Epoch 3/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0330 - mean_absolute_error: 0.1476 - val_loss: 0.0513 - val_mean_absolute_error: 0.1821
Epoch 4/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0302 - mean_absolute_error: 0.1407 - val_loss: 0.0495 - val_mean_absolute_error: 0.1789
Epoch 5/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0271 - mean_absolute_error: 0.1323 - val_loss: 0.0493 - val_mean_absolute_error: 0.1782
Epoch 6/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0247 - mean_absolute_error: 0.1242 - val_los

Epoch 50/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0048 - mean_absolute_error: 0.0405 - val_loss: 0.0547 - val_mean_absolute_error: 0.1867


 <font color='red'>
After 50 epochs:
 loss: 0.0048 - mean_absolute_error: 0.0405 - val_loss: 0.0547 - val_mean_absolute_error: 0.1867
    </font>

# CNN+LSTM

In [118]:
def define_model2(dropout,learning_rate,em,em_dim,lstm_out, n_hidden_layer,em_trainable_flag,n_filters=150):
   
    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    
    c=Dropout(0.5)(context)
    
    c=Conv1D(n_filters,kernel_size=3,activation='relu')(context)
    
    m = Bidirectional(LSTM(150,return_sequences=False,recurrent_dropout=dropout))(c)
    
    #a=  Attention()(m)
                   
    a=Dense(400,activation='relu')(m)
        
    out=Dense(1,activation='sigmoid')(a)
    
    model= Model(inputs=input_context ,outputs=out)
    

    optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    
    print(model.summary())
    
    return model

In [119]:
cnn_model_1 = define_model2(dropout=0.2,
                     learning_rate=0.01,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)
                         

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Context (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_37 (Embedding)     (None, 11, 300)           997200    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 9, 150)            135150    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 300)               361200    
_________________________________________________________________
dense_38 (Dense)             (None, 400)               120400    
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 401       
Total params: 1,614,351
Trainable params: 617,151
Non-trainable params: 997,200
______________________________________________________________

In [120]:
cnn_model_1.fit(x=input_context,y=trainY, epochs=50,batch_size=64,validation_data=(v_sentence,v_sentiment))

    

Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 7s 6ms/step - loss: 0.0415 - mean_absolute_error: 0.1799 - val_loss: 0.0465 - val_mean_absolute_error: 0.1793
Epoch 2/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.0399 - mean_absolute_error: 0.1728 - val_loss: 0.0461 - val_mean_absolute_error: 0.1790
Epoch 3/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.0393 - mean_absolute_error: 0.1707 - val_loss: 0.0464 - val_mean_absolute_error: 0.1785
Epoch 4/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.0386 - mean_absolute_error: 0.1709 - val_loss: 0.0465 - val_mean_absolute_error: 0.1780
Epoch 5/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.0376 - mean_absolute_error: 0.1662 - val_loss: 0.0461 - val_mean_absolute_error: 0.1774
Epoch 6/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.0363 - mean_absolute_error: 0.1651 - val_loss

Epoch 50/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.0011 - mean_absolute_error: 0.0210 - val_loss: 0.0539 - val_mean_absolute_error: 0.1829


relu and sigmoid : <font color='red'>
After 50 epochs:
    loss: 0.0011 - mean_absolute_error: 0.0210 - val_loss: 0.0539 - val_mean_absolute_error: 0.1829
    
</font>

# CNN +LSTM +ATTENTION

In [30]:
def define_model3(dropout,learning_rate,em,em_dim,lstm_out, n_hidden_layer,em_trainable_flag,n_filters=150):
   
    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    
    c=Dropout(0.5)(context)
    
    c=Conv1D(n_filters,kernel_size=3,activation='relu')(context)
    
    m = Bidirectional(LSTM(150,return_sequences=True,recurrent_dropout=dropout))(c)
    
    a=  Attention()(m)
                   
    a=Dense(400,activation='relu')(a)
        
    out=Dense(1,activation='sigmoid')(a)
    
    model= Model(inputs=input_context ,outputs=out)
    

    optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    
    print(model.summary())
    
    return model

In [31]:
cnn_model_3 = define_model3(dropout=0.2,
                     learning_rate=0.01,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)
                         

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Context (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 11, 300)           997200    
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 9, 150)            135150    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 9, 300)            361200    
_________________________________________________________________
attention_4 (Attention)      (None, 300)               309       
_________________________________________________________________
dense_7 (Dense)              (None, 400)               120400    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 401       
Total para

In [32]:
cnn_model_3.fit(x=input_context,y=trainY, epochs=50,batch_size=64,validation_data=(v_sentence,v_sentiment))


Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 3s 3ms/step - loss: 0.0416 - mean_absolute_error: 0.1813 - val_loss: 0.0459 - val_mean_absolute_error: 0.1795
Epoch 2/50
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0400 - mean_absolute_error: 0.1735 - val_loss: 0.0461 - val_mean_absolute_error: 0.1793
Epoch 3/50
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0396 - mean_absolute_error: 0.1728 - val_loss: 0.0461 - val_mean_absolute_error: 0.1792
Epoch 4/50
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0392 - mean_absolute_error: 0.1715 - val_loss: 0.0462 - val_mean_absolute_error: 0.1791
Epoch 5/50
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0385 - mean_absolute_error: 0.1701 - val_loss: 0.0461 - val_mean_absolute_error: 0.1787
Epoch 6/50
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0374 - mean_absolute_error: 0.1678 - val_loss

Epoch 50/50
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0013 - mean_absolute_error: 0.0234 - val_loss: 0.0536 - val_mean_absolute_error: 0.1818


After 50 epochs : loss: 0.0013 - mean_absolute_error: 0.0234 - val_loss: 0.0536 - val_mean_absolute_error: 0.1818

# 5 CHANNEL CNN MODEL

In [20]:
def cnn_model2(dropout,learning_rate,em,em_dim,em_trainable_flag,n_filters=100):
    

    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    inputs1= embedding(input_context)
    
    
    conv1 = Conv1D(filters=n_filters, kernel_size=3, activation='relu')(inputs1)
    drop1 = Dropout(dropout)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    conv2 = Conv1D(filters=n_filters, kernel_size=4, activation='relu')(inputs1)
    drop2 = Dropout(dropout)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    
    conv3 = Conv1D(filters=n_filters, kernel_size=5, activation='relu')(inputs1)
    drop3 = Dropout(dropout)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    
    conv4 = Conv1D(filters=n_filters, kernel_size=2, activation='relu')(inputs1)
    drop4 = Dropout(dropout)(conv4)
    pool4 = MaxPooling1D(pool_size=2)(drop4)
    flat4 = Flatten()(pool4)
    
    conv5 = Conv1D(filters=n_filters, kernel_size=6, activation='relu')(inputs1)
    drop5 = Dropout(dropout)(conv5)
    pool5 = MaxPooling1D(pool_size=2)(drop5)
    flat5 = Flatten()(pool5)
    # merge
    
    merged = concatenate([flat1, flat2, flat3,flat4,flat5])
    # interpretation
    dense1 = Dense(400, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=input_context, outputs=outputs)
    
    # compile
    
    optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [21]:
cnn_model_2 = cnn_model2(dropout=0.5,
                     learning_rate=0.001,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

In [22]:
cnn_model_2.fit(x=input_context,y=trainY, epochs=50,batch_size=64,validation_data=(v_sentence,v_sentiment))


Train on 1173 samples, validate on 192 samples
Epoch 1/50
1173/1173 [==============================] - 2s 1ms/step - loss: 0.0441 - mean_absolute_error: 0.1794 - val_loss: 0.0472 - val_mean_absolute_error: 0.1807
Epoch 2/50
1173/1173 [==============================] - 1s 962us/step - loss: 0.0380 - mean_absolute_error: 0.1667 - val_loss: 0.0481 - val_mean_absolute_error: 0.1798
Epoch 3/50
1173/1173 [==============================] - 1s 944us/step - loss: 0.0348 - mean_absolute_error: 0.1592 - val_loss: 0.0468 - val_mean_absolute_error: 0.1783
Epoch 4/50
1173/1173 [==============================] - 1s 973us/step - loss: 0.0311 - mean_absolute_error: 0.1488 - val_loss: 0.0467 - val_mean_absolute_error: 0.1777
Epoch 5/50
1173/1173 [==============================] - 1s 942us/step - loss: 0.0281 - mean_absolute_error: 0.1408 - val_loss: 0.0468 - val_mean_absolute_error: 0.1771
Epoch 6/50
1173/1173 [==============================] - 1s 971us/step - loss: 0.0259 - mean_absolute_error: 0.1341 

1173/1173 [==============================] - 1s 1ms/step - loss: 0.0038 - mean_absolute_error: 0.0479 - val_loss: 0.0500 - val_mean_absolute_error: 0.1742
Epoch 50/50
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0040 - mean_absolute_error: 0.0488 - val_loss: 0.0497 - val_mean_absolute_error: 0.1733


<font color='red'>
After 50 epochs:
    loss: 0.0040 - mean_absolute_error: 0.0488 - val_loss: 0.0497 - val_mean_absolute_error: 0.1733
</font>

# ASPECT MODELS


# Attention Aspect

In [20]:
no_of_classes=27

In [94]:
def define_model(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,em_dim):
     
    input_context= Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(max_length,),name='Target')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    v_target = embedding(input_target)
    
    print("Target" ,v_target.shape)
    
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(context)
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(H)
    H_all, H_last , _ = LSTM (lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="AT")(H)
    
    concat = concatenate(inputs =[v_target,H_all])
    print("Concat",concat.shape)
    
    r=AttentionWithContext(name='Attention')([H_all,concat])
    
    pooling= GlobalAveragePooling1D()(H_all)
    
    out=Final2(name='Final')([r , pooling])
    
    out=Dense(int((2*lstm_out+27)/2),activation='relu')(out)
        
    out=Dense(27,activation='softmax')(out)
    
    AT_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    AT_model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    
    
    print(AT_model.summary())
    
    return AT_model

In [96]:
At_model = define_model(learning_rate=0.00069,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

Target (?, 11, 300)
Concat (?, 11, 600)
M1 (?, 11, 600)
M (?, 11, 1)
alpha (?, 11, 1)
r (?, 300)
H_all (?, 300)
m2.shape (?, 300)
m1  (?, 300)
(?, 300) h_final
OUT (?, 27)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_28 (Embedding)        (None, 11, 300)      997200      Context[0][0]                    
                                                                 Target[0][0]                     
__________________________________________________________________________________________________
lstm_52 (LSTM)                  (None, 11, 300)      721200      embedding_28[0][0]               
____________________________________

In [93]:
Aspect_model = At_model.fit(x=trainX,y=aspect, epochs=50,batch_size=64)


Epoch 1/50
1173/1173 [==============================] - 22s 19ms/step - loss: 2.8569 - acc: 0.3478
Epoch 2/50
1173/1173 [==============================] - 13s 11ms/step - loss: 2.4569 - acc: 0.3725
Epoch 3/50
1173/1173 [==============================] - 13s 11ms/step - loss: 2.3666 - acc: 0.3751
Epoch 4/50
1173/1173 [==============================] - 13s 11ms/step - loss: 2.3366 - acc: 0.3819
Epoch 5/50
1173/1173 [==============================] - 12s 11ms/step - loss: 2.2396 - acc: 0.3845
Epoch 6/50
1173/1173 [==============================] - 12s 10ms/step - loss: 2.1441 - acc: 0.4160
Epoch 7/50
1173/1173 [==============================] - 13s 11ms/step - loss: 2.0165 - acc: 0.4348
Epoch 8/50
1173/1173 [==============================] - 13s 11ms/step - loss: 1.9025 - acc: 0.4638
Epoch 9/50
1173/1173 [==============================] - 13s 11ms/step - loss: 1.8038 - acc: 0.4962
Epoch 10/50
1173/1173 [==============================] - 12s 10ms/step - loss: 1.6637 - acc: 0.5320
Epoch 11/

After 50 epochs : loss: 0.1233 - acc: 0.9625

#  Attention + bidirectional LSTM (Adding Attention to our model)

In [74]:
def define_model_1(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,em_dim):
     
    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
        
    a = Bidirectional(LSTM(300, return_sequences=True,recurrent_dropout=dropout))(context)
    
    print(a.shape)

    alpha = Attention()(a)
    
    x=Dense(int((2*lstm_out+27)/2),activation='relu')(alpha)
        
    out=Dense(27,activation='softmax')(x)
    
    model= Model(inputs=input_context ,outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    
    
    print(model.summary())
    
    return model

In [75]:
Aspect_model_1 = define_model_1(learning_rate=0.00069,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

(?, ?, 600)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Context (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_21 (Embedding)     (None, 11, 300)           997200    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 11, 600)           1442400   
_________________________________________________________________
attention_3 (Attention)      (None, 600)               611       
_________________________________________________________________
dense_16 (Dense)             (None, 313)               188113    
_________________________________________________________________
dense_17 (Dense)             (None, 27)                8478      
Total params: 2,636,802
Trainable params: 1,639,602
Non-trainable params: 997,200
________________________________________________

In [76]:
Aspect_model = Aspect_model_1.fit(x=input_context,y=aspect, epochs=50,batch_size=64)


Epoch 1/50
1173/1173 [==============================] - 12s 10ms/step - loss: 2.8559 - acc: 0.3495
Epoch 2/50
1173/1173 [==============================] - 6s 5ms/step - loss: 2.4349 - acc: 0.3725
Epoch 3/50
1173/1173 [==============================] - 6s 5ms/step - loss: 2.2613 - acc: 0.3896
Epoch 4/50
1173/1173 [==============================] - 6s 5ms/step - loss: 2.0446 - acc: 0.4408
Epoch 5/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.8361 - acc: 0.4962
Epoch 6/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.6395 - acc: 0.5388
Epoch 7/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.4278 - acc: 0.5899
Epoch 8/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.3024 - acc: 0.6257
Epoch 9/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.1509 - acc: 0.6743
Epoch 10/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.0441 - acc: 0.6965
Epoch 11/50
1173/1173 [====

## <font color='red'>
 After 50 epochs: loss: 0.0416 - acc: 0.9889
  </font>


# ATAE

In [103]:
def Model_ATAE(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim):
    
    
    input_context= Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(max_length,),name='Target')
  
   
    #print(input_target.shape)
    #print(input_context.shape)
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    v_target = embedding(input_target)
    
    #print(context.shape)
    print("Target" ,v_target.shape)

    
    concat = concatenate(inputs =[v_target,context])
    print(concat.shape)
    inputs = Dropout(dropout)(concat)
    print(inputs.shape)   
    
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(inputs)
    H = LSTM (lstm_out, recurrent_dropout=dropout,return_sequences=True)(H)
    H_all, H_last , _ = LSTM (lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="ATAE")(H)
    
    concat = concatenate(inputs =[v_target,H_all])
    print("Concat",concat.shape)
    
    r=AttentionWithContext(name='Attention')([H_all,concat])
    
    pooling= GlobalAveragePooling1D()(H_all)
    
    out=Final2(name='Final')([r , pooling])
    
    #out=Dense(int((2*lstm_out+27)/2),activation='relu')(out)
    
    out= Activation('tanh')(out)
        
    out=Dense(27,activation='softmax')(out)
    
    ATAE_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    ATAE_model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    
    
    print(ATAE_model.summary())
    
    return ATAE_model

In [104]:
ATAE_model =Model_ATAE(learning_rate=0.00069,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

Target (?, 11, 300)
(?, 11, 600)
(?, 11, 600)
Concat (?, 11, 600)
M1 (?, 11, 600)
M (?, 11, 1)
alpha (?, 11, 1)
r (?, 300)
H_all (?, 300)
m2.shape (?, 300)
m1  (?, 300)
(?, 300) h_final
OUT (?, 27)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Target (InputLayer)             (None, 11)           0                                            
__________________________________________________________________________________________________
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_31 (Embedding)        (None, 11, 300)      997200      Context[0][0]                    
                                                                 Target[0][0]                     
__________

In [105]:
Aspect_model_3= ATAE_model.fit(x=trainX,y=aspect, epochs=50,batch_size=64)



Epoch 1/50
1173/1173 [==============================] - 30s 26ms/step - loss: 2.8263 - acc: 0.3512
Epoch 2/50
1173/1173 [==============================] - 18s 16ms/step - loss: 2.4954 - acc: 0.3734
Epoch 3/50
1173/1173 [==============================] - 19s 16ms/step - loss: 2.3471 - acc: 0.3939
Epoch 4/50
1173/1173 [==============================] - 19s 16ms/step - loss: 2.1848 - acc: 0.4246
Epoch 5/50
1173/1173 [==============================] - 19s 16ms/step - loss: 2.0342 - acc: 0.4757
Epoch 6/50
1173/1173 [==============================] - 19s 16ms/step - loss: 1.9132 - acc: 0.5038
Epoch 7/50
1173/1173 [==============================] - 19s 16ms/step - loss: 1.8269 - acc: 0.5200
Epoch 8/50
1173/1173 [==============================] - 18s 16ms/step - loss: 1.7552 - acc: 0.5362
Epoch 9/50
1173/1173 [==============================] - 18s 15ms/step - loss: 1.6784 - acc: 0.5482
Epoch 10/50
1173/1173 [==============================] - 17s 15ms/step - loss: 1.5979 - acc: 0.5908
Epoch 11/

KeyboardInterrupt: 

### After  30 epochs : loss: 0.7511 - acc: 0.7630  relu and no tanh after final layer



# IAN 

In [112]:

def Model_IAN_1_aspect(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim):
    
    input_context = Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(11,),name='Target')
    
    print("Target ",input_target.shape)
    print("Context" ,input_context.shape)
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)

    context=embedding(input_context)
    target= embedding(input_target)
    print("Target_embedding",target.shape)
    print("Context_embedding", context.shape)
 
    H_c , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_C")(context)
    H_t , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_T")(target)

    print("hc", H_c.shape)
    print("ht",H_t.shape)
    
    c_avg = GlobalAveragePooling1D(name='POOL_C')(H_c)  
    t_avg = GlobalAveragePooling1D(name='POOL_T')(H_t)
    
    print("C_AVG", c_avg.shape)
    print("t_avg",t_avg.shape)
    

    c_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_c,t_avg])
    t_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_t,c_avg])
    
    print("c_r",c_r.shape)
    print("t_r",t_r.shape)
   
    d = concatenate(inputs=[c_r , t_r])
    
    print("d",d.shape)
    
    x=Dense(int((2*lstm_out+27)/2),activation='relu')(d)
        
    out=Dense(27,activation='softmax')(x)
   
    IAN_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    IAN_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    print( IAN_model.summary())
    
    return  IAN_model
   


In [113]:
IAN_model_1=Model_IAN_1_aspect(learning_rate=0.01,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

Target  (?, 11)
Context (?, 11)
Target_embedding (?, 11, 300)
Context_embedding (?, 11, 300)
hc (?, ?, 300)
ht (?, ?, 300)
C_AVG (?, 300)
t_avg (?, 300)
c_r (?, 300)
t_r (?, 300)
d (?, 600)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
Target (InputLayer)             (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, 11, 300)      997200      Context[0][0]                    
                                                                 Target[0][0]                     
__________________

In [114]:
IAN_model_1 = IAN_model_1.fit(x=trainX,y=aspect, epochs=50,batch_size=64)


Epoch 1/50
1173/1173 [==============================] - 21s 18ms/step - loss: 2.6849 - acc: 0.3564
Epoch 2/50
1173/1173 [==============================] - 12s 11ms/step - loss: 2.4885 - acc: 0.3785
Epoch 3/50
1173/1173 [==============================] - 13s 11ms/step - loss: 2.2855 - acc: 0.3964
Epoch 4/50
1173/1173 [==============================] - 13s 11ms/step - loss: 2.1460 - acc: 0.4203
Epoch 5/50
1173/1173 [==============================] - 13s 11ms/step - loss: 1.9835 - acc: 0.4510
Epoch 6/50
1173/1173 [==============================] - 14s 12ms/step - loss: 1.8017 - acc: 0.4910
Epoch 7/50
1173/1173 [==============================] - 14s 12ms/step - loss: 1.6241 - acc: 0.5448
Epoch 8/50
1173/1173 [==============================] - 13s 11ms/step - loss: 1.5103 - acc: 0.5797
Epoch 9/50
1173/1173 [==============================] - 14s 12ms/step - loss: 1.4060 - acc: 0.5703
Epoch 10/50
1173/1173 [==============================] - 14s 12ms/step - loss: 1.2116 - acc: 0.6411
Epoch 11/

# After 50 epochs: loss: 0.0464 - acc: 0.9864

In [116]:
IAN_model_1.save('IAN_MODEL_ASPECT.h5')


AttributeError: 'History' object has no attribute 'save'

# CNN+BIDIRECTIONAL LSTM

In [117]:
def define_model2(dropout,learning_rate,em,em_dim,lstm_out, n_hidden_layer,em_trainable_flag,n_filters=150):
   
    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    
    c=Dropout(0.5)(context)
    
    c=Conv1D(n_filters,kernel_size=3,activation='relu')(context)
    
    m = Bidirectional(LSTM(150,return_sequences=False,recurrent_dropout=dropout))(c)
                   
    x=Dense(int((2*lstm_out+27)/2),activation='relu')(m)
        
    out=Dense(27,activation='softmax')(x)
   
    model= Model(inputs=input_context,outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    print(model.summary())
    
    return model

In [118]:
cnn_model_1 = define_model2(dropout=0.2,
                     learning_rate=0.01,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)
                         

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Context (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_35 (Embedding)     (None, 11, 300)           997200    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 9, 150)            135150    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 300)               361200    
_________________________________________________________________
dense_39 (Dense)             (None, 313)               94213     
_________________________________________________________________
dense_40 (Dense)             (None, 27)                8478      
Total params: 1,596,241
Trainable params: 599,041
Non-trainable params: 997,200
______________________________________________________________

In [119]:
cnn_model_1.fit(x=input_context,y=aspect, epochs=50,batch_size=64)

    

Epoch 1/50
1173/1173 [==============================] - 16s 14ms/step - loss: 2.6735 - acc: 0.3257
Epoch 2/50
1173/1173 [==============================] - 5s 4ms/step - loss: 2.1954 - acc: 0.3887
Epoch 3/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.8084 - acc: 0.4817
Epoch 4/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.4574 - acc: 0.5772
Epoch 5/50
1173/1173 [==============================] - 6s 5ms/step - loss: 1.1826 - acc: 0.6539
Epoch 6/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.8284 - acc: 0.7698
Epoch 7/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.5568 - acc: 0.8338
Epoch 8/50
1173/1173 [==============================] - 6s 5ms/step - loss: 0.3825 - acc: 0.8764
Epoch 9/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.2260 - acc: 0.9352
Epoch 10/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.1814 - acc: 0.9454
Epoch 11/50
1173/1173 [====

## After 50 epochs: loss: 0.0078 - acc: 0.9949

# CNN +LSTM +ATTENTION

In [120]:
def define_model3(dropout,learning_rate,em,em_dim,lstm_out, n_hidden_layer,em_trainable_flag,n_filters=150):
   
    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
    
    c=Dropout(0.5)(context)
    
    c=Conv1D(n_filters,kernel_size=3,activation='relu')(context)
    
    m = Bidirectional(LSTM(150,return_sequences=True,recurrent_dropout=dropout))(c)
    
    a=  Attention()(m)
                    
    x=Dense(int((2*lstm_out+27)/2),activation='relu')(a)
        
    out=Dense(27,activation='softmax')(x)
   
    model= Model(inputs=input_context,outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    print(model.summary())
    
    return model

In [121]:
cnn_model_3 = define_model3(dropout=0.2,
                     learning_rate=0.01,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)
                         

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Context (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_36 (Embedding)     (None, 11, 300)           997200    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 9, 150)            135150    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 9, 300)            361200    
_________________________________________________________________
attention_4 (Attention)      (None, 300)               309       
_________________________________________________________________
dense_41 (Dense)             (None, 313)               94213     
_________________________________________________________________
dense_42 (Dense)             (None, 27)                8478      
Total para

In [ ]:
cnn_model_3.fit(x=input_context,y=aspect, epochs=50,batch_size=64)


### After 50 epochs: loss: 0.0075 - acc: 0.9940

# 5 CHANNEL CNN MODEL

In [128]:
def cnn_model2(dropout,learning_rate,em,em_dim,em_trainable_flag,n_filters=100):
    

    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    inputs1= embedding(input_context)
    
    
    conv1 = Conv1D(filters=n_filters, kernel_size=3, activation='relu')(inputs1)
    drop1 = Dropout(dropout)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    conv2 = Conv1D(filters=n_filters, kernel_size=4, activation='relu')(inputs1)
    drop2 = Dropout(dropout)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    
    conv3 = Conv1D(filters=n_filters, kernel_size=5, activation='relu')(inputs1)
    drop3 = Dropout(dropout)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    
    conv4 = Conv1D(filters=n_filters, kernel_size=2, activation='relu')(inputs1)
    drop4 = Dropout(dropout)(conv4)
    pool4 = MaxPooling1D(pool_size=2)(drop4)
    flat4 = Flatten()(pool4)
    
    conv5 = Conv1D(filters=n_filters, kernel_size=6, activation='relu')(inputs1)
    drop5 = Dropout(dropout)(conv5)
    pool5 = MaxPooling1D(pool_size=2)(drop5)
    flat5 = Flatten()(pool5)
    # merge
    
    merged = concatenate([flat1, flat2, flat3,flat4,flat5])
    # interpretation
                   
    x=Dense(400,activation='relu')(merged)
        
    out=Dense(27,activation='softmax')(x)
   
    model= Model(inputs=input_context,outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    print(model.summary())
    
    return model
    

In [129]:
cnn_model_2 = cnn_model2(dropout=0.5,
                     learning_rate=0.001,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_39 (Embedding)        (None, 11, 300)      997200      Context[0][0]                    
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 9, 100)       90100       embedding_39[0][0]               
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 8, 100)       120100      embedding_39[0][0]               
__________________________________________________________________________________________________
conv1d_15 

In [131]:
cnn_model_2.fit(x=input_context,y=aspect, epochs=50,batch_size=64)


Epoch 1/50
1173/1173 [==============================] - 11s 9ms/step - loss: 2.5941 - acc: 0.3598
Epoch 2/50
1173/1173 [==============================] - 2s 2ms/step - loss: 1.9396 - acc: 0.4612
Epoch 3/50
1173/1173 [==============================] - 2s 2ms/step - loss: 1.3537 - acc: 0.6351
Epoch 4/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.8588 - acc: 0.7818
Epoch 5/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.4964 - acc: 0.8909
Epoch 6/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.2531 - acc: 0.9531
Epoch 7/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.1288 - acc: 0.9847
Epoch 8/50
1173/1173 [==============================] - 3s 2ms/step - loss: 0.0931 - acc: 0.9829
Epoch 9/50
1173/1173 [==============================] - 3s 2ms/step - loss: 0.0734 - acc: 0.9898
Epoch 10/50
1173/1173 [==============================] - 2s 2ms/step - loss: 0.0607 - acc: 0.9906
Epoch 11/50
1173/1173 [=====

### After 50 epochs:  loss: 0.0214 - acc: 0.9957

# CNN with 3 channels

In [23]:
def cnn_4(dropout,learning_rate,em,em_dim,em_trainable_flag,num_filters=100):
    
  
    filter_sizes = [3,4,5]


    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    inputs1= embedding(input_context)
    
    reshape = Reshape((max_length,em_dim,1))(inputs1)

    c0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], em_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
    c1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], em_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
    c2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], em_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

    DP0 = MaxPooling2D(pool_size=(3, 1), strides=(1,1), padding='valid')(c0)
    DP1 = MaxPooling2D(pool_size=(4, 1), strides=(1,1), padding='valid')(c1)
    DP2 = MaxPooling2D(pool_size=(5, 1), strides=(1,1), padding='valid')(c2)

    CT = Concatenate(axis=1)([DP0, DP1, DP2])

    flatten = Flatten()(CT)
    dropout = Dropout(dropout)(flatten)

    x=Dense(400,activation='relu')(dropout)
        
    out=Dense(27,activation='softmax')(x)
   
    model= Model(inputs=input_context,outputs=out)
    
    checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

    
    optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    
    #optimizer = Adam(lr=learning_rate)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    print(model.summary())
    
    return model

    

In [151]:
cnn_model_4 = cnn_4(dropout=0.2,
                     learning_rate=0.001,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Context (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_47 (Embedding)        (None, 11, 300)      997200      Context[0][0]                    
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 11, 300, 1)   0           embedding_47[0][0]               
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 9, 1, 100)    90100       reshape_7[0][0]                  
__________________________________________________________________________________________________
conv2d_11 

In [152]:
cnn_model_4.fit(x=input_context,y=aspect, epochs=50,batch_size=64)


Epoch 1/50
1173/1173 [==============================] - 6s 5ms/step - loss: 2.8157 - acc: 0.3350
Epoch 2/50
1173/1173 [==============================] - 1s 784us/step - loss: 2.5351 - acc: 0.3725
Epoch 3/50
1173/1173 [==============================] - 1s 847us/step - loss: 2.4314 - acc: 0.3734
Epoch 4/50
1173/1173 [==============================] - 1s 816us/step - loss: 2.3218 - acc: 0.3768
Epoch 5/50
1173/1173 [==============================] - 1s 921us/step - loss: 2.2259 - acc: 0.3930
Epoch 6/50
1173/1173 [==============================] - 1s 856us/step - loss: 2.1346 - acc: 0.4126
Epoch 7/50
1173/1173 [==============================] - 1s 885us/step - loss: 2.0219 - acc: 0.4373
Epoch 8/50
1173/1173 [==============================] - 1s 929us/step - loss: 1.9134 - acc: 0.4757
Epoch 9/50
1173/1173 [==============================] - 1s 878us/step - loss: 1.7896 - acc: 0.5072
Epoch 10/50
1173/1173 [==============================] - 1s 1ms/step - loss: 1.6998 - acc: 0.5413
Epoch 11/50
1

### After 50 epochs:  loss: 0.0837 - acc: 0.9906